In [1]:
from __future__ import print_function
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.cross_validation import train_test_split
pd.set_option('display.max_columns', None)


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


In [3]:
data = pd.read_csv('adult.dat',header=None ,na_values=' ?', names= ['age','workclass','fnlwgt','education','edu-num','marital-status',
                                                    'occupation',
                                                    'relationship','race','sex','capital-gain','capital-loss','hours-per-week',
                                                    'native-country','class'])
                   
                  
        
                   

In [4]:
data.dropna(how='any')

,age,workclass,fnlwgt,education,edu-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [5]:
data = pd.get_dummies(data)

In [6]:
data

,age,fnlwgt,edu-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,education_ 10th,education_ 11th,education_ 12th,education_ 1st-4th,education_ 5th-6th,education_ 7th-8th,education_ 9th,education_ Assoc-acdm,education_ Assoc-voc,education_ Bachelors,education_ Doctorate,education_ HS-grad,education_ Masters,education_ Preschool,education_ Prof-school,education_ Some-college,marital-status_ Divorced,marital-status_ Married-AF-spouse,marital-status_ Married-civ-spouse,marital-status_ Married-spouse-absent,marital-status_ Never-married,marital-status_ Separated,marital-status_ Widowed,occupation_ Adm-clerical,occupation_ Armed-Forces,occupation_ Craft-repair,occupation_ Exec-managerial,occupation_ Farming-fishing,occupation_ Handlers-cleaners,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male,native-country_ Cambodia,native-country_ Canada,native-country_ China,native-country_ Columbia,native-country_ Cuba,native-country_ Dominican-Republic,native-country_ Ecuador,native-country_ El-Salvador,native-country_ England,native-country_ France,native-country_ Germany,native-country_ Greece,native-country_ Guatemala,native-country_ Haiti,native-country_ Holand-Netherlands,native-country_ Honduras,native-country_ Hong,native-country_ Hungary,native-country_ India,native-country_ Iran,native-country_ Ireland,native-country_ Italy,native-country_ Jamaica,native-country_ Japan,native-country_ Laos,native-country_ Mexico,native-country_ Nicaragua,native-country_ Outlying-US(Guam-USVI-etc),native-country_ Peru,native-country_ Philippines,native-country_ Poland,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,class_ <=50K,class_ >50K
0,39,77516,13,2174,0,40,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,50,83311,13,0,0,13,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
2,38,215646,9,0,0,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,53,234721,7,0,0,40,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,28,338409,13,0,0,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
5,37,284582,14,0,0,40,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
6,49,160187,5,0,0,16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0

In [7]:
y = data.values[:,104:106]

del data['class_ <=50K'], data['class_ >50K']


X = data.values
X = preprocessing.minmax_scale(X)

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,
                                                                  test_size=0.22)

In [9]:
X[0]

array([0.30136986, 0.0443019 , 0.8       , 0.02174022, 0.        ,
       0.39795918, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [10]:
type(dat)

NameError: name 'dat' is not defined

Lots of missing data, keep columns as Q? is that a category? Think about it but maybe
Could also skip those

Either way let's set up ANN using TF!

In [11]:
# hyperparameters, adjust these like a knob
learning_rate = 0.1
num_steps = 500
batch_size = 500
display_step = 1

# Network Parameters, can play with these too, def should be larger based on this being a huge dataset. 
n_hidden_1 = 10 # 1st layer number of neurons
n_hidden_2 = 8 # 2nd layer number of neurons


num_input = len(X[0])# will know this number after I figure out how to deal with the dummies + missing data
num_classes = len(y[0]) # over or under 50k a year

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])



In [12]:
# Store layers weight & biasm only 2 layers at the moment but seemingly not hard to fix
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [13]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with a specifid amoutn of neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    activation_1 = tf.nn.relu(layer_1)
    #transitions occur after applying an activation, ooooh
    layer_2 = tf.add(tf.matmul(activation_1, weights['h2']), biases['b2'])
    activation_2 = tf.nn.sigmoid(layer_2)
    
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(activation_2, weights['out']) + biases['out']
    return out_layer

In [14]:

# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))   #Loss function ignore softmax but crossentropy is a categorization loss func good for this prob
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [15]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size,X_train,y_train)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test,
                                      Y: y_test}))
    
    
    
#Run through to make a NN, take note of input labels and classes and adjust names accordingly.

Step 1, Minibatch Loss= 0.3749, Training Accuracy= 0.612
Step 2, Minibatch Loss= 0.0513, Training Accuracy= 0.744
Step 3, Minibatch Loss= 0.0126, Training Accuracy= 0.754
Step 4, Minibatch Loss= 0.0065, Training Accuracy= 0.744
Step 5, Minibatch Loss= 0.0036, Training Accuracy= 0.758
Step 6, Minibatch Loss= 0.0033, Training Accuracy= 0.754
Step 7, Minibatch Loss= 0.0036, Training Accuracy= 0.772
Step 8, Minibatch Loss= 0.0043, Training Accuracy= 0.778
Step 9, Minibatch Loss= 0.0051, Training Accuracy= 0.736
Step 10, Minibatch Loss= 0.0164, Training Accuracy= 0.776
Step 11, Minibatch Loss= 0.0182, Training Accuracy= 0.780
Step 12, Minibatch Loss= 0.0121, Training Accuracy= 0.726
Step 13, Minibatch Loss= 0.0179, Training Accuracy= 0.768
Step 14, Minibatch Loss= 0.0253, Training Accuracy= 0.772
Step 15, Minibatch Loss= 0.0360, Training Accuracy= 0.784
Step 16, Minibatch Loss= 0.0484, Training Accuracy= 0.764
Step 17, Minibatch Loss= 0.0628, Training Accuracy= 0.732
Step 18, Minibatch Loss

Step 191, Minibatch Loss= 0.0863, Training Accuracy= 0.806
Step 192, Minibatch Loss= 0.0812, Training Accuracy= 0.766
Step 193, Minibatch Loss= 0.0798, Training Accuracy= 0.758
Step 194, Minibatch Loss= 0.0787, Training Accuracy= 0.746
Step 195, Minibatch Loss= 0.0833, Training Accuracy= 0.734
Step 196, Minibatch Loss= 0.0813, Training Accuracy= 0.746
Step 197, Minibatch Loss= 0.0853, Training Accuracy= 0.774
Step 198, Minibatch Loss= 0.0845, Training Accuracy= 0.760
Step 199, Minibatch Loss= 0.0845, Training Accuracy= 0.754
Step 200, Minibatch Loss= 0.0857, Training Accuracy= 0.762
Step 201, Minibatch Loss= 0.0883, Training Accuracy= 0.796
Step 202, Minibatch Loss= 0.0801, Training Accuracy= 0.722
Step 203, Minibatch Loss= 0.0834, Training Accuracy= 0.750
Step 204, Minibatch Loss= 0.0862, Training Accuracy= 0.782
Step 205, Minibatch Loss= 0.0831, Training Accuracy= 0.762
Step 206, Minibatch Loss= 0.0814, Training Accuracy= 0.750
Step 207, Minibatch Loss= 0.0831, Training Accuracy= 0.7

Step 377, Minibatch Loss= 0.0854, Training Accuracy= 0.742
Step 378, Minibatch Loss= 0.0824, Training Accuracy= 0.744
Step 379, Minibatch Loss= 0.0869, Training Accuracy= 0.778
Step 380, Minibatch Loss= 0.0863, Training Accuracy= 0.774
Step 381, Minibatch Loss= 0.0843, Training Accuracy= 0.758
Step 382, Minibatch Loss= 0.0883, Training Accuracy= 0.756
Step 383, Minibatch Loss= 0.0843, Training Accuracy= 0.766
Step 384, Minibatch Loss= 0.0857, Training Accuracy= 0.734
Step 385, Minibatch Loss= 0.0842, Training Accuracy= 0.764
Step 386, Minibatch Loss= 0.0882, Training Accuracy= 0.758
Step 387, Minibatch Loss= 0.0845, Training Accuracy= 0.716
Step 388, Minibatch Loss= 0.0879, Training Accuracy= 0.786
Step 389, Minibatch Loss= 0.0847, Training Accuracy= 0.760
Step 390, Minibatch Loss= 0.0834, Training Accuracy= 0.748
Step 391, Minibatch Loss= 0.0851, Training Accuracy= 0.768
Step 392, Minibatch Loss= 0.0960, Training Accuracy= 0.798
Step 393, Minibatch Loss= 0.0838, Training Accuracy= 0.8

In [ ]:
data